<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/AdaptDeepSEAvariableInputLength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
import keras

keras.__version__

Using TensorFlow backend.


'2.3.1'

In [3]:
import torch
import torch.nn as nn

from functools import reduce
from torch.autograd import Variable

class LambdaBase(nn.Sequential):
    def __init__(self, fn, *args):
        super(LambdaBase, self).__init__(*args)
        self.lambda_func = fn

    def forward_prepare(self, input):
        output = []
        for module in self._modules.values():
            output.append(module(input))
        return output if output else input

class Lambda(LambdaBase):
    def forward(self, input):
        return self.lambda_func(self.forward_prepare(input))

class LambdaMap(LambdaBase):
    def forward(self, input):
        return list(map(self.lambda_func,self.forward_prepare(input)))

class LambdaReduce(LambdaBase):
    def forward(self, input):
        return reduce(self.lambda_func,self.forward_prepare(input))

def get_model(load_weights = True):
    # alphabet seems to be fine:
    """
    https://github.com/davek44/Basset/tree/master/src/dna_io.py#L145-L148
    seq = seq.replace('A','0')
    seq = seq.replace('C','1')
    seq = seq.replace('G','2')
    seq = seq.replace('T','3')
    """
    pretrained_model_reloaded_th = nn.Sequential( # Sequential,
        nn.Conv2d(4,300,(19, 1)),
        nn.BatchNorm2d(300),
        nn.ReLU(),
        nn.MaxPool2d((3, 1),(3, 1)),
        nn.Conv2d(300,200,(11, 1)),
        nn.BatchNorm2d(200),
        nn.ReLU(),
        nn.MaxPool2d((4, 1),(4, 1)),
        nn.Conv2d(200,200,(7, 1)),
        nn.BatchNorm2d(200),
        nn.ReLU(),
        nn.MaxPool2d((4, 1),(4, 1)),
        Lambda(lambda x: x.view(x.size(0),-1)), # Reshape,
        nn.Sequential(Lambda(lambda x: x.view(1,-1) if 1==len(x.size()) else x ),nn.Linear(2000,1000)), # Linear,
        nn.BatchNorm1d(1000,1e-05,0.1,True),#BatchNorm1d,
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Sequential(Lambda(lambda x: x.view(1,-1) if 1==len(x.size()) else x ),nn.Linear(1000,1000)), # Linear,
        nn.BatchNorm1d(1000,1e-05,0.1,True),#BatchNorm1d,
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Sequential(Lambda(lambda x: x.view(1,-1) if 1==len(x.size()) else x ),nn.Linear(1000,164)), # Linear,
        nn.Sigmoid(),
    )
    if load_weights:
        sd = torch.load('/users/eprakash/pretrained_model_reloaded_th.pth')
        pretrained_model_reloaded_th.load_state_dict(sd)
    return  pretrained_model_reloaded_th

model = get_model(load_weights = True)

In [4]:
for name, param in model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([300, 4, 19, 1])
0.bias torch.Size([300])
1.weight torch.Size([300])
1.bias torch.Size([300])
4.weight torch.Size([200, 300, 11, 1])
4.bias torch.Size([200])
5.weight torch.Size([200])
5.bias torch.Size([200])
8.weight torch.Size([200, 200, 7, 1])
8.bias torch.Size([200])
9.weight torch.Size([200])
9.bias torch.Size([200])
13.1.weight torch.Size([1000, 2000])
13.1.bias torch.Size([1000])
14.weight torch.Size([1000])
14.bias torch.Size([1000])
17.1.weight torch.Size([1000, 1000])
17.1.bias torch.Size([1000])
18.weight torch.Size([1000])
18.bias torch.Size([1000])
21.1.weight torch.Size([164, 1000])
21.1.bias torch.Size([164])


In [5]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Flatten, Activation, Permute, Flatten, Dropout
from keras.layers.pooling import MaxPooling1D
from keras.layers.convolutional import Conv1D


keras_model = Sequential()
keras_model.add(Conv1D(filters=300, kernel_size=19, strides=1, padding="valid", batch_input_shape=(None, 600, 4))) #Change to 400
keras_model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
keras_model.add(Activation("relu"))
keras_model.add(MaxPooling1D(pool_size=3, strides=3, padding="valid"))
keras_model.add(Conv1D(filters=200, kernel_size=11, strides=1, padding="valid"))
keras_model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
keras_model.add(Activation("relu"))
keras_model.add(MaxPooling1D(pool_size=4, strides=4, padding="valid"))
keras_model.add(Conv1D(filters=200, kernel_size=7, strides=1, padding="valid"))
keras_model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
keras_model.add(Activation("relu"))
keras_model.add(MaxPooling1D(pool_size=4, strides=4, padding="valid"))
keras_model.add(Permute((2,1))) #Remove
keras_model.add(Flatten())
keras_model.add(Dense(1000))
keras_model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
keras_model.add(Activation("relu"))
keras_model.add(Dropout(0.3))
keras_model.add(Dense(1000))
keras_model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
keras_model.add(Activation("relu"))
keras_model.add(Dropout(0.3))
keras_model.add(Dense(164)) #Change to 1 
keras_model.add(Activation("sigmoid"))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [6]:
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 582, 300)          23100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 582, 300)          1200      
_________________________________________________________________
activation_1 (Activation)    (None, 582, 300)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 194, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 184, 200)          660200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 184, 200)          800       
_________________________________________________________________
activation_2 (Activation)    (None, 184, 200)         

In [7]:
import numpy as np

conv1_weights = (list(model.children())[0].weight.detach().cpu().numpy().squeeze()).transpose((2,1,0))
conv1_bias = (list(model.children())[0].bias.detach().cpu().numpy())
print("conv1 weights shape", conv1_weights.shape)
print("conv1 bias shape", conv1_bias.shape)
keras_model.layers[0].set_weights([conv1_weights, conv1_bias])

batchnorm1_gamma = (list(model.children())[1].weight.detach().cpu().numpy().squeeze())
batchnorm1_beta = (list(model.children())[1].bias.detach().cpu().numpy())
batchnorm1_mean = list(model.children())[1].running_mean
batchnorm1_var = list(model.children())[1].running_var.numpy().transpose()
print("batchnorm1 weights shape", batchnorm1_gamma.shape)
print("batchnorm1 bias shape", batchnorm1_beta.shape)
keras_model.layers[1].set_weights([batchnorm1_gamma, batchnorm1_beta, batchnorm1_mean, batchnorm1_var])

conv2_weights = (list(model.children())[4].weight.detach().cpu().numpy().squeeze()).transpose((2,1,0))
conv2_bias = (list(model.children())[4].bias.detach().cpu().numpy())
print("conv2 weights shape", conv2_weights.shape)
print("conv2 bias shape", conv2_bias.shape)
keras_model.layers[4].set_weights([conv2_weights, conv2_bias])

batchnorm2_gamma = (list(model.children())[5].weight.detach().cpu().numpy().squeeze())
batchnorm2_beta = (list(model.children())[5].bias.detach().cpu().numpy())
batchnorm2_mean = list(model.children())[5].running_mean
batchnorm2_var = list(model.children())[5].running_var.numpy().transpose()
print("batchnorm2 weights shape", batchnorm2_gamma.shape)
print("batchnorm2 bias shape", batchnorm2_beta.shape)
keras_model.layers[5].set_weights([batchnorm2_gamma, batchnorm2_beta, batchnorm2_mean, batchnorm2_var])

conv3_weights = (list(model.children())[8].weight.detach().cpu().numpy().squeeze()).transpose((2,1,0))
conv3_bias = (list(model.children())[8].bias.detach().cpu().numpy())
print("conv3 weights shape", conv3_weights.shape)
print("conv3 bias shape", conv3_bias.shape)
keras_model.layers[8].set_weights([conv3_weights, conv3_bias])

batchnorm3_gamma = (list(model.children())[9].weight.detach().cpu().numpy().squeeze())
batchnorm3_beta = (list(model.children())[9].bias.detach().cpu().numpy())
batchnorm3_mean = list(model.children())[9].running_mean
batchnorm3_var = list(model.children())[9].running_var.numpy().transpose()
print("batchnorm3 weights shape", batchnorm3_gamma.shape)
print("batchnorm3 bias shape", batchnorm3_beta.shape)
keras_model.layers[9].set_weights([batchnorm3_gamma, batchnorm3_beta, batchnorm3_mean, batchnorm3_var])

conv1 weights shape (19, 4, 300)
conv1 bias shape (300,)
batchnorm1 weights shape (300,)
batchnorm1 bias shape (300,)
conv2 weights shape (11, 300, 200)
conv2 bias shape (200,)
batchnorm2 weights shape (200,)
batchnorm2 bias shape (200,)
conv3 weights shape (7, 200, 200)
conv3 bias shape (200,)
batchnorm3 weights shape (200,)
batchnorm3 bias shape (200,)


In [8]:
dense1_weights = (list(list(model.children())[13].children())[1].weight.detach().cpu().numpy().squeeze()).transpose((1,0))
dense1_bias = (list(list(model.children())[13].children())[1].bias.detach().cpu().numpy())
#dense1_weights = dense1_weights[500:1500,:]
#dense1_weights = dense1_weights.reshape((200,5,1000)).transpose((1,0,2)).reshape((1000,1000))
print("dense1 weights shape", dense1_weights.shape)
print("dense1 bias shape", dense1_bias.shape)
keras_model.layers[14].set_weights([dense1_weights, dense1_bias])

batchnorm4_gamma = (list(model.children())[14].weight.detach().cpu().numpy().squeeze())
batchnorm4_beta = (list(model.children())[14].bias.detach().cpu().numpy())
batchnorm4_mean = list(model.children())[14].running_mean
batchnorm4_var = list(model.children())[14].running_var.numpy().transpose()
print(batchnorm4_var.shape)
print("batchnorm4 weights shape", batchnorm4_gamma.shape)
print("batchnorm4 bias shape", batchnorm4_beta.shape)
keras_model.layers[15].set_weights([batchnorm4_gamma, batchnorm4_beta, batchnorm4_mean, batchnorm4_var])

dense2_weights = (list(list(model.children())[17].children())[1].weight.detach().cpu().numpy().squeeze()).transpose((1,0))
dense2_bias = (list(list(model.children())[17].children())[1].bias.detach().cpu().numpy())
print("dense2 weights shape", dense2_weights.shape)
print("dense2 bias shape", dense2_bias.shape)
keras_model.layers[18].set_weights([dense2_weights, dense2_bias])

batchnorm5_gamma = (list(model.children())[18].weight.detach().cpu().numpy().squeeze())
batchnorm5_beta = (list(model.children())[18].bias.detach().cpu().numpy())
batchnorm5_mean = list(model.children())[18].running_mean
batchnorm5_var = list(model.children())[18].running_var.numpy().transpose()
print(batchnorm5_var.shape)
print("batchnorm5 weights shape", batchnorm5_gamma.shape)
print("batchnorm5 bias shape", batchnorm5_beta.shape)
keras_model.layers[19].set_weights([batchnorm5_gamma, batchnorm5_beta, batchnorm5_mean, batchnorm5_var])

dense3_weights = (list(list(model.children())[21].children())[1].weight.detach().cpu().numpy().squeeze()).transpose((1,0))
dense3_bias = (list(list(model.children())[21].children())[1].bias.detach().cpu().numpy())
print("dense3 weights shape", dense3_weights.shape)
print("dense3 bias shape", dense3_bias.shape)
keras_model.layers[22].set_weights([dense3_weights, dense3_bias])

keras_model.build()

dense1 weights shape (2000, 1000)
dense1 bias shape (1000,)
(1000,)
batchnorm4 weights shape (1000,)
batchnorm4 bias shape (1000,)
dense2 weights shape (1000, 1000)
dense2 bias shape (1000,)
(1000,)
batchnorm5 weights shape (1000,)
batchnorm5 bias shape (1000,)
dense3 weights shape (1000, 164)
dense3 bias shape (164,)


In [9]:
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 582, 300)          23100     
_________________________________________________________________
batch_normalization_1 (Batch (None, 582, 300)          1200      
_________________________________________________________________
activation_1 (Activation)    (None, 582, 300)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 194, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 184, 200)          660200    
_________________________________________________________________
batch_normalization_2 (Batch (None, 184, 200)          800       
_________________________________________________________________
activation_2 (Activation)    (None, 184, 200)         

In [ ]:
#keras_model.save('basset.h5')

In [10]:
np.random.seed(1)
inp = np.random.random((10, 4, 600, 1))

#onehot encode
inp = (inp==np.max(inp,axis=1)[:,None,:,:]).astype("float32")
inp.shape

(10, 4, 600, 1)

In [11]:
def get_pytorch_preds(list_of_layers, the_input):
    for layer in list_of_layers:
        the_input = layer.eval().forward(the_input)
    return the_input

pytorch_predictions = get_pytorch_preds(
    list_of_layers=list(model.children()),
    the_input=torch.from_numpy(inp.astype("float32"))).detach().cpu().numpy()

In [12]:
#intermediate_keras=keras.Model(inputs=keras_model.input, outputs=keras_model.layers[1].output)
keras_preds = keras_model.predict(inp.squeeze().transpose((0, 2, 1)))

In [13]:
pytorch_predictions.shape

(10, 164)

In [14]:
keras_preds.shape

(10, 164)

In [15]:
np.max(np.abs(pytorch_predictions-keras_preds))

7.1525574e-07